# 할 일

1. 구별 클러스터링
2. 클러스터별 면적 계산
3. 사고발생건수/면적 계산: 기준 이상만 남기기

In [1]:
import warnings
import pyproj
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium import plugins
from statsmodels.api import OLS
from sklearn.cluster import DBSCAN
from functools import partial
from shapely.ops import transform
from shapely.geometry import Point


Bad key "text.kerning_factor" on line 4 in
/home/jhgan/anaconda3/envs/accident/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
warnings.filterwarnings(action="ignore")

In [3]:
proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')
def geodesic_point_buffer(point, meter):
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=point.y, lon=point.x)),
        proj_wgs84
    )
    buf = Point(0, 0).buffer(meter)  # distance in metres
    return transform(project, buf)

In [4]:
def calc_meter_area(polygon):
    proj = partial(
        pyproj.transform,
        pyproj.Proj(init='epsg:4326'),
        pyproj.Proj(init='epsg:3857')
    )
    s_new = transform(proj, polygon)
    return s_new.area

In [5]:
data = pd.read_json("data/kids-accident-pp.json")
data = data.assign(
    sido = [x.split()[0] for x in data.legaldong_name],
    gugun = [x.split()[1] for x in data.legaldong_name],
    acdnt_dd_dc = pd.to_datetime(data.acdnt_dd_dc, format="%Y-%m-%d")
)

In [6]:
gdf = gpd.read_file("data/geometry.geojson")
gdf = gdf.assign(rad_X = np.radians(gdf.geometry.x), rad_Y=np.radians(gdf.geometry.y))

# 1. 기준 설정

In [7]:
acdnt_cnt = (data.groupby(data.acdnt_dd_dc.dt.year)
                 .size()
                 .reset_index()
                 .rename(columns={"acdnt_dd_dc":"year", 0:"cnt"})
                 .assign(constant=1)
            )

In [8]:
ols_model = OLS(acdnt_cnt.cnt.values,acdnt_cnt[["constant","year"]]).fit()
ols_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.957
Model:                            OLS   Adj. R-squared:                  0.953
Method:                 Least Squares   F-statistic:                     245.5
Date:                Mon, 15 Jun 2020   Prob (F-statistic):           7.20e-09
Time:                        23:22:28   Log-Likelihood:                -91.486
No. Observations:                  13   AIC:                             187.0
Df Residuals:                      11   BIC:                             188.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
constant    7.054e+05   4.47e+04     15.787      0.000    6.07e+05    8.04e+05
year        -347.7637     22.197    -15.667      0.000    -396.620    -298.908
==============================================================================
Omnibus:                        0.697   Durbin-Watson:                   0.847
Prob(Omnibus):                  0.706   Jarque-Bera (JB):                0.015
Skew:                          -0.076   Prob(JB):                        0.992
Kurtosis:                       3.071   Cond. No.                     1.08e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.08e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [9]:
summation = 2
criterion = 2
print("year","|","criterion")
print("====","|","="*20)
print(acdnt_cnt.year.values[-1],"|",criterion)
for i in range(acdnt_cnt.shape[0]-1):
    criterion = criterion/ols_model.fittedvalues.values[-(i+1)] * ols_model.fittedvalues.values[-(i+2)]
    summation += criterion
    print(acdnt_cnt.year.values[-(i+2)], "|", criterion)

year | criterion
==== | ====================
2019 | 2
2018 | 2.212058873785856
2017 | 2.4241177475717115
2016 | 2.636176621357638
2015 | 2.848235495143494
2014 | 3.0602943689293496
2013 | 3.272353242715205
2012 | 3.484412116501132
2011 | 3.6964709902869877
2010 | 3.9085298640728436
2009 | 4.120588737858699
2008 | 4.332647611644555
2007 | 4.544706485430481


In [10]:
# 2건 X 13년 = 26
criterion = 26 / (300 * 300 * np.pi)

# 2. 클러스터 선정

In [11]:
seoul = data.query("sido=='서울특별시'").drop("sido", axis=1)

In [27]:
earth_radius_km = 6371
epsilon = 0.1 / earth_radius_km #calculate 150 meter epsilon threshold
min_samples = int(26/9)

In [28]:
result = []
for gu in seoul.gugun.unique():
    print(gu)
    # 클러스터 모델 적합
    df = seoul[seoul.gugun==gu]
    geo = gdf[gdf.acdnt_no.isin(df.acdnt_no)].drop("acdnt_no",axis=1)
    model = DBSCAN(
        eps=epsilon,
        min_samples=min_samples,
        n_jobs=6
    )
    model.fit(geo[["rad_X", "rad_Y"]])
    if model.labels_.max()==-1:
        print("-----------------------------------")
        print("클러스터 없음")
        print("-----------------------------------\n")
        continue
    before = pd.Series(model.labels_).value_counts().sort_index().reset_index().rename(columns={"index":"cluster", 0:"count"})
    
    # 밀도 계산
    geo = geo.assign(
        labels = model.labels_
    ).query("labels!=-1").assign(geometry = geo.geometry.apply(geodesic_point_buffer, meter=100))
    
    geo = pd.merge(
        geo.dissolve(by="labels").reset_index(),
        geo.labels.value_counts().reset_index().rename(columns={"labels":"cls_size", "index":"labels"})
    )
    geo = geo.assign(meter_area = geo.geometry.apply(calc_meter_area))
    geo = geo.assign(density = geo.cls_size/geo.meter_area)

    print("-----------------------------------")
    print(
        before.assign(
            alive = before.cluster.isin(geo[geo.density > criterion].labels),
            density = [0] + list(geo.sort_values("labels").density)
        )
    )
    print("-----------------------------------\n")
    result.append(geo.copy())

도봉구
-----------------------------------
    cluster  count  alive   density
0        -1     93  False  0.000000
1         0      4  False  0.000050
2         1      2  False  0.000028
3         2     14  False  0.000077
4         3      8  False  0.000043
..      ...    ...    ...       ...
79       78      5  False  0.000042
80       79      2  False  0.000040
81       80      2  False  0.000035
82       81      2  False  0.000038
83       82      2  False  0.000029

[84 rows x 4 columns]
-----------------------------------

강남구
-----------------------------------
    cluster  count  alive   density
0        -1    192  False  0.000000
1         0      2  False  0.000026
2         1      7  False  0.000051
3         2      4  False  0.000071
4         3      4  False  0.000063
..      ...    ...    ...       ...
88       87      2  False  0.000029
89       88      3  False  0.000038
90       89      2  False  0.000028
91       90      2  False  0.000040
92       91      2  False  0.000

-----------------------------------
     cluster  count  alive   density
0         -1    222  False  0.000000
1          0     12  False  0.000074
2          1      2  False  0.000028
3          2     11  False  0.000050
4          3      8  False  0.000074
..       ...    ...    ...       ...
137      136      2  False  0.000034
138      137      2  False  0.000026
139      138      3  False  0.000055
140      139      2  False  0.000026
141      140      3  False  0.000043

[142 rows x 4 columns]
-----------------------------------

성북구
-----------------------------------
     cluster  count  alive   density
0         -1    130  False  0.000000
1          0      9  False  0.000056
2          1     10  False  0.000061
3          2     13  False  0.000063
4          3      3  False  0.000042
..       ...    ...    ...       ...
107      106      2  False  0.000030
108      107      3  False  0.000037
109      108      2  False  0.000033
110      109      2  False  0.000034
111      110

-----------------------------------
     cluster  count  alive   density
0         -1    110  False  0.000000
1          0      4  False  0.000058
2          1      4  False  0.000036
3          2     11  False  0.000085
4          3      5  False  0.000063
..       ...    ...    ...       ...
97        96      2  False  0.000030
98        97      2  False  0.000035
99        98      3  False  0.000031
100       99      3  False  0.000042
101      100      2  False  0.000040

[102 rows x 4 columns]
-----------------------------------

중구
-----------------------------------
    cluster  count  alive   density
0        -1     57  False  0.000000
1         0      2  False  0.000026
2         1      4  False  0.000040
3         2      4  False  0.000038
4         3      5  False  0.000060
5         4      8  False  0.000059
6         5      3  False  0.000036
7         6      3  False  0.000029
8         7      5  False  0.000059
9         8      2  False  0.000031
10        9      3  Fals

# 3. 지도 시각화

In [29]:
cluster = pd.concat(result).query(f"density > {criterion}")
acdnt_cls = gpd.sjoin(gdf[['geometry', 'acdnt_no']], cluster[['labels', 'geometry']],  op="within")
acdnt_cls = pd.merge(acdnt_cls, data)
acdnt_cls = acdnt_cls.assign(acdnt_dd_dc = acdnt_cls.acdnt_dd_dc.astype(str))

In [30]:
m = folium.Map(location=[37.53, 126.97], zoom_start=12) 

folium.features.GeoJson(cluster).add_to(m)
plugins.ScrollZoomToggler().add_to(m)
folium.features.GeoJson(acdnt_cls).add_to(m)

In [33]:
m

In [32]:
m.save("tmp.html")